# Lattice Green function

In this notebook, we will first manipulate a bit the Green function on a lattice, 

$$ G(k, i\omega_n) $$
where $k$ is a vector in the Brillouin Zone.


In [ ]:
%matplotlib inline
from pytriqs.plot.mpl_interface import plt
import numpy as np
from math import cos, pi
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct

In [ ]:
beta = 1./0.4
#mu = 0.0
t = 1.0 

#n_k = (8, 8, 1) 
n_w = 128

# A mesh on a Brillouin Zone

We first need a simple Bravais Lattice in 2 dimensions,  its corresponding Brillouin Zone and a mesh on it.

In [ ]:
# Two unit vectors in R3
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)])

BZ = BrillouinZone(BL)



The Green function $$ G(k, i\omega_n) $$
is on a cartesian product mesh : $ (k \times i\omega_n) \rightarrow {\mathcal{C}}$

In [ ]:
# Construct the Green function

# A regular mesh on the BZ with 30 x 30 points
kmesh = MeshBrillouinZone(BZ, n_k = 30)

# Frequency mesh
wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)

g = Gf(mesh = MeshProduct(kmesh, wmesh),target_shape = [])

print g, g.rank, g.data.shape


In [ ]:
iGamma = 0.01 * 1j

def eps(k):
    return -2 * t* (cos(k[0]) + cos(k[1]))

# NB : loop is a bit slow in python ...
for k in g.mesh[0]:
    for w in g.mesh[1]:
        g[k,w] = 1/(w - eps(k) + iGamma)

In [ ]:
k = [0.0, 0., 0]
g(k, 8) 

In [ ]:
mf = lambda n : (2*n+1)*pi/beta*1j
for k in g.mesh[0]:
    assert abs(g(k.value, 0) - 1/(mf(0) - eps(k) + iGamma))< 1.e-14

In [ ]:
# NOW HERE we plot G(k, om)
# We show how to slice for fixed k, fixed omega
# and make a SIMPLE plot


# Fix FIX the evaluator of the GF to be periodic
# We use it on a grid on BZ ...